In [141]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from collections import Counter
import random
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [7]:
principal_axes = np.load('Principal components.npy')
# The data contains 70,000 28 × 28 images (60,000 as training and 10,000 as testing)

fashion_mnist_train = pd.read_csv('fashion-mnist_train.csv')
print('The shape for the fashion_mnist_train is', fashion_mnist_train.shape)

fashion_mnist_test = pd.read_csv('fashion-mnist_test.csv')
print('The shape for the fashion_mnist_test is', fashion_mnist_test.shape)

The shape for the fashion_mnist_train is (60000, 785)
The shape for the fashion_mnist_test is (10000, 785)


In [8]:
X, y = fashion_mnist_train[fashion_mnist_train.columns.values[1:]],\
fashion_mnist_train['label']
print('The shape for X, y are {} and {}, respectively.'.format(X.shape, y.shape))

# Standardize all the numerical features
ss = StandardScaler()
X_std = ss.fit_transform(X)
X_std = pd.DataFrame(X_std, columns = X.columns)

The shape for X, y are (60000, 784) and (60000,), respectively.


In [9]:
X_test, y_test = fashion_mnist_test[fashion_mnist_train.columns.values[1:]],\
fashion_mnist_test['label']

## Gradient Boosting

In [151]:
# three-fold cross-validation using GridSearchCV
grid = {'max_depth': [10, 20], 'n_estimators': [20, 30, 40]}
clf = GradientBoostingClassifier(random_state = 0)
grid_search = GridSearchCV(estimator = clf, 
                           param_grid = grid,
                           cv = 3, 
                           scoring = 'accuracy')
clf_cv = grid_search.fit(X_std @ principal_axes[:, :30], y)
print('The best parameters using 3-fold cv is {}'.format(clf_cv.best_params_))

The best parameters using 3-fold cv is {'max_depth': 10, 'n_estimators': 20}


In [15]:
clf_cv.cv_results_

{'mean_fit_time': array([2.37495756, 3.58770728, 4.80095625, 3.50468032, 5.27307296,
        7.04044763]),
 'std_fit_time': array([0.01638441, 0.01432371, 0.01329186, 0.01085088, 0.02716352,
        0.06343185]),
 'mean_score_time': array([0.03805041, 0.05563021, 0.07337769, 0.06249968, 0.09274578,
        0.12721411]),
 'std_score_time': array([7.42177494e-04, 6.06622830e-04, 8.60115929e-05, 1.27124733e-03,
        2.65854682e-03, 9.96187093e-03]),
 'param_max_depth': masked_array(data=[10, 10, 10, 20, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[20, 30, 40, 20, 30, 40],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 10, 'n_estimators': 20},
  {'max_depth': 10, 'n_estimators': 30},
  {'max_depth': 10, 'n_estimators': 40},
  {'max_depth': 20, 'n_estimators': 20},
  {'max_dep

In [145]:
clf_gb = GradientBoostingClassifier(max_depth = 10, n_estimators = 20).fit(X_std @ principal_axes[:, :30], y)
y_pred_gb = clf_gb.predict(ss.fit_transform(X_test) @ principal_axes[:, :30])
cm = confusion_matrix(y_test, y_pred_gb)
correct_classification_rate = np.round(np.diagonal(cm) / np.sum(cm, axis = 0), 4)
correct_classification_rate = pd.DataFrame(data = correct_classification_rate.reshape(1, -1),
                                      columns = np.arange(0, 10), index = ['Correct Classification Rate'])
correct_classification_rate

,0,1,2,3,4,5,6,7,8,9
Correct Classification Rate,0.7249,0.9503,0.6929,0.8032,0.6994,0.8351,0.5362,0.8294,0.8851,0.8691


In [154]:
overall_correct_classification_rate = np.round(np.sum(np.diagonal(cm)) / np.sum(cm), 4)
overall_correct_classification_rate

0.8416


In [147]:
y_bd = clf_bd.predict(X_std @ principal_axes[:, :30])

In [33]:
y_knn = np.load('KNN_train.npy')
y_dt = np.load('DT_train.npy')
y_svm  = np.load('SVM_train.npy')

In [123]:
X_train = np.zeros((60000, 4))
X_train[:, 0] = y_knn
X_train[:, 1] = y_dt
X_train[:, 2] = y_svm
X_train[:, 3] = y_gb
X_train.shape

(60000, 4)

In [35]:
y_pred_knn = np.load('KNN Prediction.npy')
y_pred_dt = np.load('Decision Tree Prediction.npy')
y_pred_svm  = np.load('Support Vector Machine Prediction.npy')

In [124]:
X_test_ = np.zeros((10000, 4))
X_test_[:, 0] = y_pred_knn
X_test_[:, 1] = y_pred_dt
X_test_[:, 2] = y_pred_svm
X_test_[:, 3] = y_pred_gb
X_test_.shape

(10000, 4)

In [90]:
reg = LogisticRegression().fit(X_train, y)
reg.coef_

/Users/yujingyang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([[ -3.46118181, -68.4537173 ,   9.47712779, -65.40124411],
       [ -1.32373226, -13.71715318,   3.9543691 , -11.94875737],
       [  1.01787578,  -4.16741627,   1.07561651,   1.91970973],
       [  0.89594886,   5.34579655,  -1.51856856,   5.5052231 ],
       [  0.67599711,   8.73653702,  -1.20049644,   6.7887306 ],
       [  3.44524224,   7.47466298,  -1.72580339,  10.307729  ],
       [ -1.31171263,  18.03846599,  -2.93538866,   9.2701789 ],
       [  3.86014775,  11.49962223,  -1.97956106,  11.51652517],
       [ -1.71723221,  16.86430748,  -2.6912022 ,  15.2694324 ],
       [ -2.08135283,  18.37889449,  -2.45609311,  16.77247258]])

In [157]:
y_pred_ensemble = reg.predict(X_test_)
cm = confusion_matrix(y_test, y_pred_ensemble)
correct_classification_rate = np.round(np.diagonal(cm) / np.sum(cm, axis = 0), 4)
correct_classification_rate = pd.DataFrame(data = correct_classification_rate.reshape(1, -1),
                                      columns = np.arange(0, 10), index = ['Correct Classification Rate'])
correct_classification_rate

,0,1,2,3,4,5,6,7,8,9
Correct Classification Rate,0.957079,0.994284,0.797997,0.887863,0.883063,0.865737,0.698785,0.942709,0.985546,0.996143


In [152]:
overall_correct_classification_rate = np.round(np.sum(np.diagonal(cm)) / np.sum(cm), 4)
overall_correct_classification_rate

0.8843


In [161]:
random.seed(1)
for i in range(10000):
    values = [y_pred_knn[i], y_pred_dt[i], y_pred_svm[i]]
    count = dict(Counter(values))
    mv = max(count.values())
    y_pred_ensemble[i] = random.choice([k for (k, v) in count.items() if v == mv])
cm = confusion_matrix(y_test, y_pred_ensemble)
correct_classification_rate = np.round(np.diagonal(cm) / np.sum(cm, axis = 0), 4)
correct_classification_rate = pd.DataFrame(data = correct_classification_rate.reshape(1, -1),
                                      columns = np.arange(0, 10), index = ['Correct Classification Rate'])
correct_classification_rate

,0,1,2,3,4,5,6,7,8,9
Correct Classification Rate,0.839716,0.952371,0.842383,0.904299,0.835443,0.937855,0.785954,0.906044,0.939694,0.917601


In [153]:
overall_correct_classification_rate = np.round(np.sum(np.diagonal(cm)) / np.sum(cm), 4)
overall_correct_classification_rate

0.8872
